##Demo of Avalance for continual learning

Install Avalanche

In [7]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
!pip install torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
!pip install avalanche-lib==0.2.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 582 kB 4.6 MB/s 
  Attempting uninstall: avalanche-lib
    Found existing installation: avalanche-lib 0.2.0
    Uninstalling avalanche-lib-0.2.0:
      Successfully uninstalled avalanche-lib-0.2.0


## Import libraries


In [12]:
from torchvision import datasets

In [5]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
# from avalanche.benchmarks.classic import SplitMNIST
from avalanche.evaluation.metrics import forgetting_metrics, accuracy_metrics, \
    loss_metrics, timing_metrics, cpu_usage_metrics, confusion_matrix_metrics, disk_usage_metrics
from avalanche.models import SimpleMLP
from avalanche.logging import InteractiveLogger, TextLogger, TensorboardLogger
from avalanche.training.plugins import EvaluationPlugin
from avalanche.training.supervised import Naive

scenario = SplitMNIST(n_experiences=5)

In [ ]:
# MODEL CREATION
model = SimpleMLP(num_classes=scenario.n_classes)

TensorBoard logging

In [ ]:

tb_logger = TensorboardLogger()

# log to text file
text_logger = TextLogger(open('log.txt', 'a'))

# print to stdout
interactive_logger = InteractiveLogger()


In [ ]:
eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    forgetting_metrics(experience=True, stream=True),
    cpu_usage_metrics(experience=True),
    confusion_matrix_metrics(num_classes=scenario.n_classes, save_image=False,
                             stream=True),
    disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loggers=[interactive_logger, text_logger, tb_logger]
)

/usr/local/lib/python3.7/dist-packages/avalanche/training/plugins/evaluation.py:82: UserWarning: No benchmark provided to the evaluation plugin. Metrics may be computed on inconsistent portion of streams, use at your own risk.
  "No benchmark provided to the evaluation plugin. "


In [ ]:
# CREATE THE STRATEGY INSTANCE (NAIVE)
cl_strategy = Naive(
    model, SGD(model.parameters(), lr=0.001, momentum=0.9),
    CrossEntropyLoss(), train_mb_size=500, train_epochs=1, eval_mb_size=100,
    evaluator=eval_plugin)

In [ ]:
# TRAINING LOOP
print('Starting experiment...')
results = []
for experience in scenario.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
    results.append(cl_strategy.eval(scenario.test_stream))


Starting experiment...
Start of experience:  0
Current Classes:  [0, 5]
-- >> Start of training phase << --
100%|██████████| 23/23 [00:06<00:00,  3.69it/s]
Epoch 0 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 55644.3730
	DiskUsage_MB/train_phase/train_stream/Task000 = 55644.3730
	Loss_Epoch/train_phase/train_stream/Task000 = 1.2534
	Loss_MB/train_phase/train_stream/Task000 = 0.4151
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0112
	Time_Epoch/train_phase/train_stream/Task000 = 6.2123
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.6499
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.9128
-- >> End of training phase << --
Training completed
Computing accuracy on the whole test set
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 19/19 [00:00<00:00, 27.21it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	CPUUsage_Exp/eval_phase/test_stream/Task000/Exp000 = 85.8873
	DiskUsage_Exp/eval